In [52]:
#
# import libariry
#
import numpy as np
import pandas as pd
# data precession
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
# model
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from helper import utility
import importlib
importlib.reload(utility)


<module 'helper.utility' from 'c:\\Users\\Lewkh\\Documents\\GitHub\\CE4041_ML\\helper\\utility.py'>

In [53]:
prop_2016 = utility.load_data('data/properties_2016.csv')
prop_2017 = utility.load_data('data/properties_2017.csv')

c:\Users\Lewkh\Documents\GitHub\CE4041_ML\helper\utility.py:7: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
c:\Users\Lewkh\Documents\GitHub\CE4041_ML\helper\utility.py:7: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [54]:
utility.print_percent_missing(prop_2016)

parcelid : 0.0%

fips : 0.38312122703307666%

latitude : 0.38312122703307666%

longitude : 0.38312122703307666%

propertylandusetypeid : 0.38312122703307666%

rawcensustractandblock : 0.38312122703307666%

regionidcounty : 0.38312122703307666%

assessmentyear : 0.38318822383766404%

bedroomcnt : 0.3835567062628948%

bathroomcnt : 0.38395868709041925%

roomcnt : 0.3843941663202374%

propertycountylandusecode : 0.4112598849597868%

regionidzip : 0.46830766406596236%

taxamount : 1.0468250716782062%

taxvaluedollarcnt : 1.4253570175970458%

structuretaxvaluedollarcnt : 1.8418091549123563%

calculatedfinishedsquarefeet : 1.8613387234495853%

yearbuilt : 2.0074922526570096%

regionidcity : 2.10520709214774%

landtaxvaluedollarcnt : 2.268947282559358%

censustractandblock : 2.5167014659235827%

calculatedbathnbr : 4.31834603648579%

fullbathcnt : 4.31834603648579%

finishedsquarefeet12 : 9.24666448033761%

lotsizesquarefeet : 9.248875374888994%

propertyzoningdesc : 33.71908976801352%

unitc